In [113]:
## here i will generate the input for bypass from the vcf file generates by slim 
## Baypass needs 3 data input to run the core + the covariates (environmental) model 

# 1) genotyping data file
# 2) pool (haploid) size file 
# 3) covariate data file

In [14]:
#2025
1800/5

360.0

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import math 
import os

import allel
import subprocess

In [132]:
#check when used in snakemake 
path = 'arq_pi0.001_beta'

In [206]:
## The genotyping data file
## matrix with nsnp rows and 2 ∗ npop columns
## The row field separator is a space
## each row corresponds to one marker and the number of columns is
## twice the number of populations because each pair of numbers corresponds to
## each allele (or read counts for PoolSeq experiment) counts in one population

## so basically is need a ref and alt column for each pop in each snp 
allele_counts = pd.read_csv('../test/arq_pi0.001_beta5_allele_counts_filtered.csv',header=None,index_col=0)
columns_names = pd.read_csv('../test/arq_pi0.001_beta5_allele_counts.csv',nrows=1).columns[1:]
allele_counts.columns = columns_names

In [210]:
allele_counts = allele_counts.astype(int)

In [217]:
allele_counts.to_csv( path + '/geno_table', header=None, sep = ' ', index=False)

In [216]:
allele_counts

,ref_optima10_subp6.vcf,alt_optima10_subp6.vcf,ref_optima10_subp1.vcf,alt_optima10_subp1.vcf,ref_optima10_subp8.vcf,alt_optima10_subp8.vcf,ref_optima10_subp9.vcf,alt_optima10_subp9.vcf,ref_optima10_subp0.vcf,alt_optima10_subp0.vcf,...,ref_optima6_subp1.vcf,alt_optima6_subp1.vcf,ref_optima6_subp8.vcf,alt_optima6_subp8.vcf,ref_optima6_subp9.vcf,alt_optima6_subp9.vcf,ref_optima6_subp0.vcf,alt_optima6_subp0.vcf,ref_optima6_subp7.vcf,alt_optima6_subp7.vcf
0,,,,,,,,,,,,,,,,,,,,,
62,370,24,224,10,352,14,340,60,296,6,...,0,400,0,400,0,400,0,400,0,400
188,370,24,224,10,352,14,340,60,296,6,...,200,0,200,0,200,0,200,0,200,0
269,24,370,10,224,14,352,60,340,6,296,...,200,0,200,0,200,0,200,0,200,0
591,370,24,224,10,352,14,340,60,296,6,...,200,0,200,0,200,0,200,0,200,0
1155,0,394,0,234,0,366,0,400,0,302,...,0,400,0,400,0,400,0,400,0,400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26866522,197,0,117,0,183,0,200,0,151,0,...,0,400,0,400,0,400,0,400,0,400
26917201,197,0,117,0,183,0,200,0,151,0,...,0,400,0,400,0,400,0,400,0,400
26933021,197,0,117,0,183,0,200,0,151,0,...,0,400,0,400,0,400,0,400,0,400


In [ ]:
# 2.3.2 The pool (haploid) size file [only required for Pool–Seq data]
#  (twice the number of pooled individuals for diploid species) of each population should be provided
# each population in each columns, if there are 20 population, then there should be 20 columns 

In [3]:
pool_sizes = pd.read_csv('../test/arq_pi0.001_beta5_indiv_number.csv', index_col= 0)

In [4]:
## multiply so we get the pool haploid size 
pool_sizes = pool_sizes.astype(int) * 2

In [194]:
pool_sizes.to_csv( path +'/pool_size', header=None, sep = ' ', index=False)

In [11]:
len(pool_sizes.T)

180

In [10]:
# d0yij for 
pool_sizes.T[0].min() / 5

30.0

In [163]:
# 2.3.3 The covariate data file [required for the covariate modes]
# populations (columns) and covariates (row)
# Note that in most cases, it is (strongly) recommended to scale each covariate (so that
# µˆ = 0 and σc2 = 1 

## extraction fo the env variable: 
columns_names = pd.read_csv('../test/arq_pi0.001_beta5_allele_counts.csv',nrows=1).columns[1:]

env_var_mapper = pd.read_csv('/Users/tbellagio/tbellagio_cluster/simulations/snakemake/results/arq_pi0.001_beta5/optima_files/optima.csv', usecols=[2])

## get the columnname, ONE PER EACH SUBP AND OPTIMA, becuase they are duplciated based on alt/ref allele counts
## adn then just extract the optima index to then map that to the actual value of the env variable 
env_var_index = pd.Series(columns_names.str[4:]).drop_duplicates().str.split('_').str[0].str.replace('optima', '').astype(int)



In [171]:
env_var = pd.DataFrame(env_var_index).rename(columns={0:'bio1'})


In [173]:
env_var['bio1'] = env_var['bio1'].replace(env_var_mapper['bio1'])

In [165]:
# scaling since recommenede in baypass manual 

In [174]:
# Scale the column using NumPy
env_var['bio1'] = (env_var['bio1'] - np.mean(env_var['bio1'])) / np.std(env_var['bio1'])

In [175]:
# round since too many decimals (10+) 
env_var['bio1'] = env_var['bio1'].round(4)

In [176]:
env_var.T.to_csv(path + '/cov_data',header=None, sep = ' ', index=False)

In [177]:
env_var

,bio1
0,-1.9105
2,-1.9105
4,-1.9105
6,-1.9105
8,-1.9105
...,...
350,-0.5280
352,-0.5280
354,-0.5280
356,-0.5280


In [183]:
pool_size = pd.read_csv('arq_pi0.001_beta/pool_size', sep =' ')

In [220]:
geno_table = pd.read_csv('arq_pi0.001_beta/geno_table',sep =' ',header=None)

In [186]:
cov_data = pd.read_csv('arq_pi0.001_beta/cov_data',sep =' ')

In [187]:
pool_size

,394.0,234.0,366.0,400.0,302.0,150.0,400.0.1,400.0.2,400.0.3,400.0.4,...,400.0.163,400.0.164,400.0.165,400.0.166,278.0,400.0.167,400.0.168,400.0.169,400.0.170,400.0.171


In [222]:
geno_table.dtypes

0      int64
1      int64
2      int64
3      int64
4      int64
       ...  
355    int64
356    int64
357    int64
358    int64
359    int64
Length: 360, dtype: object

In [223]:
'.' in geno_table

False

In [12]:
204800/2

102400.0